In [1]:
## IMPORTS
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   

In [2]:
## TOOL FUNCTIONS
def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].Close/df.iloc[index-30].Close
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['30WMA']/df.iloc[index-1]['30WMA']



In [3]:
#*Stage Checkers
def checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL):
    stageOneIndicator = 0
    falseReason = ""
    if volumePerc<0.7:
        stageOneIndicator+=1
    else:
        falseReason += "volume "
    if slope<=1.03 and slope>=0.995:
        stageOneIndicator+=1
    else:
        falseReason += "slope "
    if P4WH<P4WL*1.15:
        stageOneIndicator+=1
    else:
        falseReason += "range "
    if price <= WMA*1.1 and price>= WMA*0.9:
        stageOneIndicator+=1
    else:
        falseReason += "price "
    if stageOneIndicator>=3:
        return True
    return "False " + falseReason
def checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage,prevClose):
    if prevStage != "Stage 2" and price > 150:
        return "Too Expansive"
    if volumePerc < 0.7:
        if prevStage != "Stage 2":
            #print(prevStage)
            return "Volume"
    if RS < 0.1 and prevStage != "Stage 2":
        return "RS"
    if RS < 0 and prevStage == "Stage 2":
        return "RS"
    if slope < 1.03 and prevStage != "Stage 2":
        return "Slope"
    if slope < 1.005 and prevStage == "Stage 2":
        return "Slope"
    if price < WMA*1.1 and prevStage != "Stage 2":
        return "Price"
    if price < WMA*0.95 and prevStage == "Stage 2":
        return "Price"
    return "Clear"
def checkStage(price,volumePerc, RS, slope, WMA,P4WH,P4WL,prevStage,prevClose):
    stage1Check = checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL)
    stage2Check = checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage,prevClose)
    if stage2Check == "Clear":
        return "Stage 2"
    if stage1Check == True:
        return "Stage 1 " + stage2Check 
    return stage2Check    

In [4]:
def returnStageDf(dfSorted,spDfSorted):
    deltaX = 10.4
    weights = np.arange(1,31)
    sumWeights = np.sum(weights)
    dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
    dfSorted['30WMASlope'] = dfSorted.apply(lambda x: calculateSlope(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
    dfSorted['P4WH'] = dfSorted['Close'].rolling(window=4).max()
    dfSorted['P4WL'] = dfSorted['Close'].rolling(window=4).min()
    dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change()
    dfSorted['Percent'] = dfSorted.apply(lambda x: product(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
    spDfSorted['Percent'] = dfSorted.apply(lambda x: product(spDfSorted,spDfSorted.index.get_loc(x.name)),axis=1)
    dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
    dfSorted = dfSorted.dropna()
    dfSorted['Stage'] = ""
    for index, element in dfSorted.iterrows():
        if dfSorted.index.get_loc(index) == 0:
            continue
        dfSorted.iloc[dfSorted.index.get_loc(index), dfSorted.columns.get_loc('Stage')] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Stage'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Close'])
    return dfSorted[["Close","Stage"]]

def getStage(ticker):
    spDfSorted = pd.read_csv('stockData/S&P500.csv').set_index('Date')
    try:
        return returnStageDf(pd.read_csv('stockData/R3000/'+ticker+'.csv').set_index('Date'),spDfSorted)
    except:
        pass
    try:
        return returnStageDf(pd.read_csv('stockData/S&P500/'+ticker+'.csv').set_index('Date'),spDfSorted)
    except:
        pass
    return 0
    
    

In [5]:
transaction = pd.read_csv('stockData/S&P500.csv').set_index('Date').drop(['Close','Volume'],axis=1)
transaction['holding'] = np.empty((len(transaction), 0)).tolist()

In [6]:
R3000TXT = open("stockData/russell3000Symbols.txt", "r")
R3000List = R3000TXT.read()
dfR3000Tickers = R3000List.split(",")
R3000TXT.close()
dfSPYTickers = pd.read_csv('stockData/S&P500Symbols.csv')
dfR3000 = list(zip(dfR3000Tickers, ['not hold']*len(dfR3000Tickers)))
dfSPY = list(zip(dfSPYTickers['Symbol'], ['not hold']*len(dfR3000Tickers)))
dfReal = []
for i in dfR3000:
    if i not in dfReal:
       dfReal.append(i)
for i in dfSPY:
    if i not in dfReal:
       dfReal.append(i)

In [7]:
totalEarnings = 0
totalCost = 0
totalSPYEarings = 0
totalTime = 0
count = 0
totalPerc = 0
SPYPerc = 0
spy = pd.read_csv('stockData/S&P500.csv').set_index('Date')
F500 = ['GM', 'F', 'XOM', 'T', 'GE', 'MO', 'DD', 'CVS', 'C', 'PG', 'PEP', 'AMCR', 'HPE', 'KR', 'MET', 'XLB', 'BA', 'TGT', 'ENNV', 'TKUMF', 'UPS', 'WINA', 'C', 'CNC', 'PYS', 'XRX', 'ETN', 'KODK', 'PCTY', 'MRO', 'COST', 'KO', 'WBREOX', 'JNJ', 'SIVB', 'AXP', 'SEAS', 'MMM', 'MRK', 'IP', 'CAT', 'VZ', 'LCID', 'NIO', 'LMT', 'BAC', 'SPMYY', 'ATVK', 'HD', 'MCK', 'COP', 'DAL', 'GT', 'BUD', 'BMY', 'JPM', 'ACI', 'SBAC', 'INTC', 'ADM', 'ROK', 'NUSI', 'CSCO', 'QREARX', 'IWM', 'WBA', 'TXN', 'WM', 'DIS', 'RTX', 'CSLT', 'MLM', 'TXT', 'ALGN', 'OXY', 'BAX', 'WBA', 'COUP', 'LEHLQ', 'ABT', 'TNA', 'TNA', 'MKFG.WT', 'ETHUSD', 'EMR', 'AOMOY', 'GIS', 'FDX', 'FBHS', 'EIX', 'MCD', 'SO', 'FL', 'PFE', 'UNP', 'WHR', 'PFG', 'CI', 'VIAC', 'CL', 'ANSS', 'KMB', 'AU', 'JNJ', 'LLY', 'OSRS', 'LNC', 'FMCKJ', 'JCI', 'DOW', 'NOC', 'WUMSY', 'CPB', 'MS', 'QCOM', 'SEE', 'DFS', 'PPG', 'XYL', 'EHC', 'ED', 'USB', 'AFL', 'LCID', 'LEVI', 'GS', 'HON', 'ETN', 'WFC', 'ICE', 'PEG', 'CAH', 'HAL', 'DDS', 'CMG', 'BTO', 'TXN', 'AEP', 'VOBIF', 'EVO', 'KEY', 'TWMTX', 'DOTUSD', 'CLR', 'ARMK', 'FB', 'TSN', 'MRZLF', 'FSRXU', 'W', 'WFC', 'RC.PRE', 'TCPIX', 'RHCO', 'RRD', 'GPC', 'GLW', 'CMI', 'R', 'PNC', 'ARW', 'MSFT', 'NSC', 'AKTX', 'LWLG', 'BKR', 'DOMR', 'DUK', 'MAS', 'CCK', 'RAD', 'EMN', 'ABC', 'MAN', 'FIBK', 'EXC', 'LRLCY', 'CNP', 'JWN', 'BRK.B', 'DLLFF', 'GESI', 'CP', 'PBI', 'NKE', 'SCL', 'UNH', 'GE', 'COF', 'COUP', 'GD', 'HUM', 'JLL', 'COF', 'UNM', 'CENX', 'CMS', 'CGNSF', 'PMCB', 'OVV', 'SFE', 'FUTU', 'RJF', 'TSM', 'DTP', 'RISR', 'PPWLM', 'STX', 'APD', 'ITW', 'THC', 'MARA', 'MPW', 'KMX', 'OC', 'HZNP', 'RIORX', 'DGII', 'JAZZ', 'SYUS', 'MET', 'LEA', 'FTMRQ', 'SHW', 'DOV', 'HRL', 'MPXOF', 'LPX', 'GWW', 'XOM', 'SPGI', 'BBY', 'NUE', 'UHS', 'ALLT', 'MODV', 'SHIP', 'FREE', 'AVY', 'BC', 'PFG', 'NURE', 'HTGC', 'AAPL', 'MGAWY', 'ELAMF', 'XM', 'KO', 'PARXF', 'PEBN', 'CAT', 'WMB', 'HAS', 'PSTH', 'WTM', 'ALGN', 'PKI', 'USO', 'DOTUSD', 'LUV', 'PH', 'TRV', 'PEI', 'BDX', 'CMA', 'F', 'BKR', 'XEL', 'RCII', 'ARVIX', 'DTRC', 'PGR', 'JNJ', 'TDY', 'USB', 'FE', 'KELYA', 'NYT', 'ACER', 'BLDP', 'HCYIX', 'SON', 'AES', 'YSACU', 'WING', 'JD', 'A', 'MCHP', 'FIJEX']
test = ['A','AAPL','GOOG','AA']
for symbol in dfSPY:
    try:
        df = getStage(symbol[0])
        shares = 0
        sharesSPY = 0
        for index, element in df.iterrows():
            if shares == 0 and element.Stage == "Stage 2":
                shares = 1000/element.Close
                sharesSPY = 1000/spy.loc[index].Close
                totalTime +=1
                totalCost +=1000
                count+=1
                transaction.loc[index]['holding'].append((symbol[0],element.Close,0))
                print("buy "+index)
                continue
            if shares != 0:
                if element.Stage == "Stage 2":
                    totalTime +=1
                    transaction.loc[index]['holding'].append((symbol[0],element.Close,0))
                else:
                    totalEarnings += shares*element.Close
                    totalSPYEarings += sharesSPY*spy.loc[index].Close
                    totalPerc += shares*element.Close/1000
                    SPYPerc += sharesSPY*spy.loc[index].Close/1000
                    print(symbol[0]+" "+str(shares*element.Close))
                    print("SPY "+ str(sharesSPY*spy.loc[index].Close))
                    print("sell "+index)
                    shares = 0
                    transaction.loc[index]['holding'].append((symbol[0],element.Close,-1))
        if shares !=0:
            totalEarnings += df.iloc[-1].Close
            totalSPYEarings += sharesSPY*spy.iloc[-1].Close
            totalPerc += shares*element.Close/1000
            SPYPerc += sharesSPY*spy.loc[index].Close/1000
            transaction.loc[index]['holding'].append((symbol[0],element.Close,-1))
    except:
        print(symbol[0]+" Stock Not Found")
print(totalPerc/count)
print(SPYPerc/count)
print(totalTime/count)
print((totalTime/500)/(10*52))
print(count)

buy 1997-10-10
ABMD 790.6976744186046
SPY 988.0246165498958
sell 1997-11-28
buy 1999-05-07
ABMD 862.081699156132
SPY 987.1227165137082
sell 1999-08-13
buy 1999-11-12
ABMD 1293.1311672022798
SPY 1073.4351626420755
sell 2000-03-31
buy 2003-06-06
ABMD 806.7092661491638
SPY 997.914475643651
sell 2003-07-04
buy 2003-09-26
ABMD 852.109330121502
SPY 1092.1001763979427
sell 2003-12-19
buy 2004-04-23
ABMD 808.4441112922273
SPY 965.6233896398626
sell 2004-07-16
buy 2015-01-30
ABMD 1423.6567698598521
SPY 1042.2909977496117
sell 2015-10-30
buy 1995-08-25
ATVI 903.1435324650123
SPY 1102.4460049704262
sell 1995-12-08
buy 2001-02-02
ATVI 1631.0737917634747
SPY 809.6067810188064
sell 2001-09-14
buy 1999-06-18
ADBE 1394.341600204688
SPY 1012.8980882310832
sell 2000-01-28
buy 2000-04-07
ADBE 891.0079956054688
SPY 936.3867428426432
sell 2000-07-28
buy 1996-11-15
AMD 1467.701470673974
SPY 1211.0165470627378
sell 1997-06-13
buy 1999-12-03
AMD 2230.15873015873
SPY 990.6439449362668
sell 2000-07-28
buy 2004-

In [8]:
# fullPrint(transaction)
transactionData = transaction.copy()
test = pd.DataFrame()
test['holding'] = ([('F',50,0)],[('F',40,0)],[('F',30,-1)],[],[('M',25,0)],[('M',30,-1)])
test['total'] = (0,0,0,0,0,0)
testData = test.copy()
#fullPrint(transactionData)

In [9]:
transaction = transactionData.copy()
total = 100
first = True
prevIndex = ''
transaction['total'] = 0
transaction = transaction.astype({'total': 'int64'})
for index, element in transaction.iterrows():
    try:
        if len(element.holding) == 0:
            transaction['total'][index] = total
            continue
        if first: 
            first = not first
            transaction['total'][index] = total
        else:
            prevData = transaction.loc[prevIndex].holding
            if len(prevData):
                total = 0
            for i in range(len(prevData)):
                for j in range(len(element.holding)):
                    if element.holding[j][0] == prevData[i][0]:
                        close = element.holding[j][1]
                total += close*prevData[i][2]
            transaction['total'][index] = total
            for k in element.holding:
                if k[2] == -1:
                    element.holding.remove(k)
        count = len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1], total/count/element.holding[i][1])
        prevIndex = index
    except:
        print(index)
        break
        

In [10]:
fullPrint(transaction)
transaction.to_csv('Testdata/Weekly/'+date.today().strftime('%Y-%m-%d')+'full.csv')

                                                      holding     total
Date                                                                   
1995-01-06                                                 []       100
1995-01-13                                                 []       100
1995-01-20                                                 []       100
1995-01-27                                                 []       100
1995-02-03                                                 []       100
1995-02-10                                                 []       100
1995-02-17                                                 []       100
1995-02-24                                                 []       100
1995-03-03                                                 []       100
1995-03-10                                                 []       100
1995-03-17                                                 []       100
1995-03-24                                                 []   

In [11]:
import pandas as pd
test = pd.DataFrame()
test['holding'] = ([('F',50,0),('APPL',60,0)],[('F',40,0),('APPL',70,-1)],[('F',30,-1)],[],[('M',25,0)],[('M',30,-1)])
test['total'] = (0,0,0,0,0,0)
testData = test.copy()
test = testData.copy()
total = 100
first = True
prevIndex = ''
test['total'] = 0
test = test.astype({'total': 'int64'})
print(test.dtypes)
for index, element in test.iterrows():
    if len(element.holding) == 0:
        test['total'][index] = total
        continue
    if first: 
        first = not first
        test['total'][index] = total
    else:
        prevData = test.loc[prevIndex].holding
        if len(prevData):
            total = 0
        for i in range(len(prevData)):
            for j in range(len(element.holding)):
                if element.holding[j][0] == prevData[i][0]:
                    close = element.holding[j][1]
            total += close*prevData[i][2]
        test['total'][index] = total
        for k in element.holding:
            if k[2] == -1:
                element.holding.remove(k)
    count = len(element.holding)
    for i in range(len(element.holding)):
        element.holding[i] = (element.holding[i][0],element.holding[i][1], total/count/element.holding[i][1])
    prevIndex = index
        

holding    object
total       int64
dtype: object


In [12]:
print(test)

                                          holding  total
0  [(F, 50, 1.0), (APPL, 60, 0.8333333333333334)]    100
1                   [(F, 40, 2.4583333333333335)]     98
2                                              []     73
3                                              []     73
4                                 [(M, 25, 2.95)]     73
5                                              []     88
